In [ ]:
# Import necessary libraries
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

import math
import datetime
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# start time
now = datetime.datetime.now()
print (now.strftime("%Y-%m-%d %H:%M:%S"))
# load yield strength data
df = pd.read_csv('huang-401.csv')
#df = df[df.ys.notnull()]
df = df[df.phase.notnull()]
df


2023-12-30 10:31:55


FileNotFoundError: ignored

In [ ]:
df.info()


In [ ]:
le = LabelEncoder()
df['phase'] = le.fit_transform(df['phase'].to_numpy())
df.sample(10)


In [ ]:
# input data
X = df[['vec', 'delta', 'deltachi', 'deltahmix', 'deltasmix']]
# output data
Y = df['phase']
X, Y


In [ ]:
seed = 123
# data split
from sklearn.model_selection import train_test_split
## train - 80%
## test - 20%
# split the data for train, calibration and test
x_train, x_remain, y_train, y_remain = train_test_split(X, Y, test_size=0.25, random_state=seed)
x_cal, x_test, y_cal, y_test = train_test_split(x_remain, y_remain, test_size=0.50, random_state=seed)
# x_train.describe(), x_cal.describe(), x_test.describe()
y_cal


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Tạo mô hình Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)


In [ ]:
# predict on calibration data
n = len(x_cal)
preds = model.predict_proba(x_cal)
prob_true = preds[np.arange(n), y_cal]
scores = 1 - prob_true
# n, preds, prob_true, scores
preds, scores


In [ ]:
# Determine the conformity threshold without using 'method' argument
alpha = 0.05
q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(scores, q_level)

# Creating prediction sets for the test data
prediction_sets = (1 - model.predict_proba(x_test) <= qhat)

# Evaluate the predictions by checking if the true label falls within the predicted set
real = le.classes_[y_test]
predset = ['']*len(y_test)
coverage = [0]*len(y_test)
for i in range(len(y_test)):
    predset[i] = ','.join(le.classes_[prediction_sets[i]])
    if real[i] in predset[i]:
        coverage[i] = 1


In [ ]:
# Calculate the coverage rate
cover_rate = sum(coverage)/len(coverage)

# Display the cover rate, qhat value, and an example of prediction sets
cover_rate, qhat, prediction_sets[:5]


In [ ]:
# result
res = pd.DataFrame(list(zip(real, predset, coverage)),
                  columns=['actual', 'predicted', 'coverage'])
res


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
# print metric to get performance
print("Accuracy: ", model.score(x_test, y_test))
yhat = model.predict(x_test)


In [ ]:
conf_matrix = confusion_matrix(y_true=y_cal, y_pred=model.predict(x_cal))
conf_matrix
